In [ ]:
!pip install flask pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading

app = Flask(__name__)

ngrok.set_auth_token("2vDawMr8nxQ2KlLxV7WlpCKBRIw_3se1tn5SGeT7Y2Gv3UUcN")

public_url = ngrok.connect(5000)
print("Public URL:", public_url)

# Question banks
questions = {
    "Data Scientist": [
        "Can you explain the bias-variance tradeoff?",
        "What is overfitting and how can you prevent it?",
        "Describe a project where you used machine learning."
    ],
    "Frontend Developer": [
        "What is the virtual DOM?",
        "Explain the difference between Flexbox and Grid.",
        "How do you optimize a React application?"
    ],
    "HR": [
        "Tell me about a time you dealt with a conflict at work.",
        "How do you handle tight deadlines?",
        "What motivates you in your job?"
    ],
    "General": [
        "Why do you want this job?",
        "How do you work in a team?",
        "Describe a challenge you overcame."
    ]
}

# Session storage
sessions = {}

def get_mixed_question(role, current_q):
    """
    Alternate questions from role-specific, HR, and General.
    e.g., Q0 -> role question, Q1 -> HR question, Q2 -> general question,
    Q3 -> next role question, and so on.
    """
    role_qs = questions.get(role, [])
    hr_qs = questions["HR"]
    gen_qs = questions["General"]

    # Calculate index in each category
    role_index = current_q // 3
    hr_index = current_q // 3
    gen_index = current_q // 3

    mod = current_q % 3

    if mod == 0 and role_index < len(role_qs):
        return role_qs[role_index]
    elif mod == 1 and hr_index < len(hr_qs):
        return hr_qs[hr_index]
    elif mod == 2 and gen_index < len(gen_qs):
        return gen_qs[gen_index]

    # If questions exhausted in a category, fallback to any remaining role questions
    remaining_role_qs = role_qs[role_index:] if role_index < len(role_qs) else []
    if remaining_role_qs:
        return remaining_role_qs[0]

    # No more questions
    return None

@app.route("/", methods=["GET"])
def home():
    return "✅ Interview Bot is running! Use POST /start and /answer endpoints."

@app.route('/start', methods=['POST'])
def start_interview():
    data = request.json
    name = data.get('name')
    role = data.get('role')

    if not name or not role:
        return jsonify({"error": "Please provide 'name' and 'role' in request."}), 400

    if role not in questions:
        # Clarifying question if role not supported
        return jsonify({
            "error": "Role not supported.",
            "message": "Can you specify your role or skill set more precisely?"
        }), 400

    session_id = f"{name}_{role}"
    sessions[session_id] = {
        "name": name,
        "role": role,
        "current_q": 0,
        "answers": []
    }
    first_question = get_mixed_question(role, 0)
    return jsonify({"question": first_question, "session_id": session_id})

@app.route('/answer', methods=['POST'])
def answer_question():
    data = request.json
    session_id = data.get('session_id')
    answer = data.get('answer')

    if not session_id or not answer:
        return jsonify({"error": "Please provide 'session_id' and 'answer'."}), 400

    if session_id not in sessions:
        return jsonify({"error": "Session not found."}), 404

    session = sessions[session_id]
    session['answers'].append(answer)
    session['current_q'] += 1

    next_q = get_mixed_question(session['role'], session['current_q'])

    if not next_q:
        return jsonify({
            "message": "Interview completed",
            "answers": session['answers']
        })

    return jsonify({"question": next_q})

def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()


Public URL: NgrokTunnel: "https://2a69-35-184-44-178.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import requests

ngrok_url = "https://1708-35-184-44-178.ngrok-free.app"  # Replace with your actual ngrok URL

# Start interview by asking for candidate name and role
name = input("Enter your name: ")
role = input("Enter the role you are applying for: ")

start_resp = requests.post(f"{ngrok_url}/start", json={
    "name": name,
    "role": role
})

if start_resp.status_code != 200:
    print("Error:", start_resp.json().get("error"))
    exit()

data = start_resp.json()
print("\nQuestion 1:", data["question"])

session_id = data["session_id"]
question_num = 1

while True:
    answer = input("Your answer: ")
    answer_resp = requests.post(f"{ngrok_url}/answer", json={
        "session_id": session_id,
        "answer": answer
    })

    if answer_resp.status_code != 200:
        print("Error:", answer_resp.json().get("error"))
        break

    data = answer_resp.json()
    if "message" in data and data["message"] == "Interview completed":
        print("\nInterview Completed! Your answers:")
        for i, ans in enumerate(data["answers"], 1):
            print(f"{i}. {ans}")
        print("\nThank you for your time! We will get back to you soon. Goodbye! 👋")
        break
    else:
        question_num += 1
        print(f"\nQuestion {question_num}:", data["question"])


Enter your name: Iram Fatima
Enter the role you are applying for: Data Scientist


INFO:werkzeug:127.0.0.1 - - [16/May/2025 12:38:32] "POST /start HTTP/1.1" 200 -



Question 1: Can you explain the bias-variance tradeoff?
Your answer: I don't know


INFO:werkzeug:127.0.0.1 - - [16/May/2025 12:38:39] "POST /answer HTTP/1.1" 200 -



Question 2: Tell me about a time you dealt with a conflict at work.
Your answer: When I was a secretary and had to organize  tech fest I had to dela with the conflict between president and vice president


...


Question 8: What motivates you in your job?
Your answer: curiosity


INFO:werkzeug:127.0.0.1 - - [16/May/2025 12:43:11] "POST /answer HTTP/1.1" 200 -



Question 9: Describe a challenge you overcame.
Your answer: my fear of public speaking


INFO:werkzeug:127.0.0.1 - - [16/May/2025 12:43:22] "POST /answer HTTP/1.1" 200 -



Interview Completed! Your answers:
1. I don't know
2. When I was a secretary and had to organize  tech fest I had to dela with the conflict between president and vice president
3. because I have a great understanding about data science and is a perfect candidate for this role
4. I don't know
5. working
6. By agreeing with everyone
7. surprise housing
8. curiosity
9. my fear of public speaking

Thank you for your time! We will get back to you soon. Goodbye! 👋
